In [6]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Load the dataset
data = pd.read_csv("/content/jokes-data.csv")

In [7]:
# Check for Missing Values
missing_values = data.isnull().sum()
print("Missing Values:\n", missing_values)

# Drop Unnecessary Columns
data.drop(columns=['id'], inplace=True)

Missing Values:
 id         0
user_id    0
joke_id    0
Rating     0
dtype: int64


In [8]:
# Check for Duplicates
duplicates = data.duplicated().sum()
print("Duplicate Rows:", duplicates)

Duplicate Rows: 0


In [9]:
print(data.head())

   user_id  joke_id  Rating
0    31030      110   2.750
1    16144      109   5.094
2    23098        6  -6.438
3    14273       86   4.406
4    18419      134   9.375


In [10]:
# Create User-Item Matrix
user_item_matrix = data.pivot_table(index='user_id', columns='joke_id', values='Rating')


In [11]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
 #Calculate Similarity
user_similarity = cosine_similarity(user_item_matrix.fillna(0))

In [ ]:
# Recommendation Generation
def recommend_jokes(user_id, user_similarity, user_item_matrix):
    similar_users = user_similarity[user_id]
    rated_jokes = user_item_matrix.loc[user_id].dropna().index
    unrated_jokes = user_item_matrix.columns.difference(rated_jokes)
    recommendations = {}
    for joke_id in unrated_jokes:
        similar_users_ratings = user_item_matrix.loc[user_similarity.argsort()[user_id, ::-1][:10], joke_id].dropna()
        if not similar_users_ratings.empty:
            weighted_rating = similar_users_ratings.dot(similar_users[:len(similar_users_ratings)])
            weighted_sum = similar_users[:len(similar_users_ratings)].sum()
            recommendations[joke_id] = weighted_rating / weighted_sum
    return sorted(recommendations.items(), key=lambda x: x[1], reverse=True)

# Example: Generate recommendations for a user
user_id = 110
recommendations = recommend_jokes(user_id, user_similarity, user_item_matrix)
print("Recommended Jokes for User", user_id)
for joke_id, rating in recommendations[:5]:
    print("Joke ID:", joke_id, "- Predicted Rating:", round(rating, 2))